In [1]:
import dynamo as dyn
import numpy as np
import pandas as pd

print(dyn.__version__)

0.0+7a67ded


/home/xqiu/.local/lib/python3.6/site-packages/dynamo-0.0+7a67ded-py3.6.egg/dynamo/plot/topography.py:492: SyntaxWarning: 'scatters.parameters.no_show_legend|kwargs' is not a valid key!
  q_kwargs_dict={},


In [2]:
# read in the splicing data 

In [3]:
neuron_splicing_4_11 = dyn.read_h5ad('/home/xqiu/proj/Aristotle/data/u_penn_data/data/revision_data/neuron_splicing_4_11.h5ad')

In [4]:
# read in the labeling data 

In [5]:
Neu = dyn.read_h5ad('/home/xqiu/proj/Aristotle/data/u_penn_data/data/revision_data/Neu_one_shot.h5ad') # Neu

In [6]:
Neu

AnnData object with n_obs × n_vars = 18133 × 37661 
    obs: 'nGene', 'nUMI', 'time', 'cluster'
    var: 'gene_short_name'
    layers: 'new', 'total'

In [7]:
# update neuron_splicing_4_11's obs 

In [8]:
# neuron_splicing.obs = Neu.obs.loc[neuron_splicing.obs.index, :]
neuron_splicing_4_11.obs = Neu.obs.loc[neuron_splicing_4_11.obs.index, :]


In [9]:
neuron_splicing_4_11

AnnData object with n_obs × n_vars = 13476 × 44021 
    obs: 'nGene', 'nUMI', 'time', 'cluster'
    var: 'gene_short_name'
    layers: 'spliced', 'unspliced'

In [10]:
adata = neuron_splicing_4_11.copy()

In [11]:
intersect_gene = list(set(adata.var_names).intersection(Neu.var_names))
intersect_cells = list(set(adata.obs_names).intersection(Neu.obs_names))

In [12]:
adata = adata[intersect_cells, :][:, intersect_gene].copy()

In [13]:
Neu = Neu[intersect_cells, :][:, intersect_gene].copy()

In [14]:
adata

AnnData object with n_obs × n_vars = 13476 × 37007 
    obs: 'nGene', 'nUMI', 'time', 'cluster'
    var: 'gene_short_name'
    layers: 'spliced', 'unspliced'

In [15]:
Neu

AnnData object with n_obs × n_vars = 13476 × 37007 
    obs: 'nGene', 'nUMI', 'time', 'cluster'
    var: 'gene_short_name'
    layers: 'new', 'total'

In [16]:
tmp = Neu.X

In [17]:
adata.X = Neu.X.copy()


In [18]:
adata.layers['unspliced'] = Neu.layers['new'].copy()

In [21]:
adata.obs['label_time'] = 2 # this is the labeling time 
tkey = 'label_time'
# adata = adata[adata.obs.time.isin(time), :]

cell_filter = (adata.X > 0).sum(1) > 2000

adata = adata[cell_filter.A1, :]

peng_gene_list = pd.read_csv('/home/xqiu/proj/Aristotle/data/u_penn_data/scNT_revision/0408_grp_info.txt', sep='\t')

adata = adata[:, peng_gene_list.index]

adata

View of AnnData object with n_obs × n_vars = 3060 × 97 
    obs: 'nGene', 'nUMI', 'time', 'cluster', 'label_time'
    var: 'gene_short_name'
    layers: 'spliced', 'unspliced'

In [25]:
adata = dyn.pp.recipe_monocle(adata, n_top_genes=1000, fc_kwargs={"min_expr_genes_s": 5,
    "min_expr_genes_u": 2,}, fg_kwargs={"shared_count": 5},
#                       keep_filtered_cells=False,
#                       keep_filtered_genes=False,
                      # total_layers='spliced',
                      )

dyn.tl.dynamics(adata, model='stochastic')

dyn.tl.reduceDimension(adata, reduction_method='umap')

dyn.tl.cell_velocities(adata, calc_rnd_vel=True)
adata.obs['time'] = adata.obs.time.astype('category')

dyn.pl.phase_portraits(adata, genes=peng_gene_list.index[0:25], color='time', discrete_continous_div_color_key_cmap=['viridis', None, None], save_show_or_return='save', pointsize=3, save_kwargs={'prefix': 'n_s_labeling_pp_0_25_filter_cells'})
dyn.pl.phase_portraits(adata, genes=peng_gene_list.index[25:50], color='time', discrete_continous_div_color_key_cmap=['viridis', None, None], save_show_or_return='save', pointsize=3, save_kwargs={'prefix': 'n_s_labeling_pp_25_50_filter_cells'})
dyn.pl.phase_portraits(adata, genes=peng_gene_list.index[50:75], color='time', discrete_continous_div_color_key_cmap=['viridis', None, None], save_show_or_return='save', pointsize=3, save_kwargs={'prefix': 'n_s_labeling_pp_50_75_filter_cells'})
dyn.pl.phase_portraits(adata, genes=peng_gene_list.index[75:], color='time', discrete_continous_div_color_key_cmap=['viridis', None, None], save_show_or_return='save', pointsize=3, save_kwargs={'prefix': 'n_s_labeling_pp_75_above_filter_cells'})

hao_gene_list = ["Egr1", "Fos", "Fosb", "Ier2", "Jun", "Nr4a3", "Homer1", "Fosl2", "Nefl", "Maff"]
dyn.pl.phase_portraits(adata, genes=hao_gene_list, color='time',  discrete_continous_div_color_key=[None, None, None], discrete_continous_div_color_key_cmap=['viridis', None, None], ncols=6, save_kwargs={'prefix': 'n_s_hao_gene_list_pp'}, save_show_or_return='save', pointsize=5)

dyn.pl.cell_wise_velocity(adata, color='time', color_key_cmap = 'viridis', basis='umap', ncols=5, quiver_size=4,
                          quiver_length=5, save_show_or_return='save', save_kwargs={'prefix': 'labeling_cell_wise_velocity_filter_cells'})  # ['GRIA3', 'LINC00982', 'AFF2']

dyn.pl.grid_velocity(adata, color='time', color_key_cmap = 'viridis', basis='umap', ncols=5, save_show_or_return='save', save_kwargs={'prefix': 'n_s_labeling_grid_velocity_filter_cells'})  # ['GRIA3', 'LINC00982', 'AFF2']
dyn.pl.streamline_plot(adata, color='time', color_key_cmap = 'viridis', basis='umap_rnd', ncols=5, show_legend='on data', save_show_or_return='save', save_kwargs={'prefix': 'n_s_labeling_streamline_rnd_filter_cells', 'ext': 'svg'})  # ['GRIA3', 'LINC00982', 'AFF2']
dyn.pl.streamline_plot(adata, color='time', color_key_cmap = 'viridis', basis='umap_rnd', ncols=5, save_show_or_return='save', save_kwargs={'prefix': 'n_s_labeling_streamline_rnd_legend_filter_cells', 'ext': 'svg'})  # ['GRIA3', 'LINC00982', 'AFF2']

dyn.pl.streamline_plot(adata, color='time', color_key_cmap = 'viridis', basis='umap', ncols=5, show_legend='on data', save_show_or_return='save', save_kwargs={'prefix': 'n_s_labeling_stream_velocity_filter_cells', 'ext': 'svg'})  # ['GRIA3', 'LINC00982', 'AFF2']
dyn.pl.streamline_plot(adata, color='time', color_key_cmap = 'viridis', basis='umap', ncols=5, save_show_or_return='save', save_kwargs={'prefix': 'n_s_labeling_stream_velocity_legend_filter_cells', 'ext': 'svg'})  # ['GRIA3', 'LINC00982', 'AFF2']


estimating gamma: 100%|██████████| 97/97 [00:00<00:00, 128.23it/s]
/home/xqiu/.local/lib/python3.6/site-packages/scipy/sparse/_index.py:118: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)
calculating density corrected drift: 100%|██████████| 3060/3060 [00:01<00:00, 2606.13it/s]
/home/xqiu/.local/lib/python3.6/site-packages/dynamo-0.0+7a67ded-py3.6.egg/dynamo/plot/dynamics.py:175: MatplotlibDeprecationWarning: 
The DivergingNorm class was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use TwoSlopeNorm instead.
  div_scatter_kwargs.update({"norm": DivergingNorm(0)})


Saving figure to /home/xqiu/proj/Aristotle/data/u_penn_data/n_s_labeling_pp_0_25_filter_cells_dyn_savefig.pdf...
Done


/home/xqiu/.local/lib/python3.6/site-packages/dynamo-0.0+7a67ded-py3.6.egg/dynamo/plot/dynamics.py:175: MatplotlibDeprecationWarning: 
The DivergingNorm class was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use TwoSlopeNorm instead.
  div_scatter_kwargs.update({"norm": DivergingNorm(0)})


Saving figure to /home/xqiu/proj/Aristotle/data/u_penn_data/n_s_labeling_pp_25_50_filter_cells_dyn_savefig.pdf...
Done


/home/xqiu/.local/lib/python3.6/site-packages/dynamo-0.0+7a67ded-py3.6.egg/dynamo/plot/dynamics.py:175: MatplotlibDeprecationWarning: 
The DivergingNorm class was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use TwoSlopeNorm instead.
  div_scatter_kwargs.update({"norm": DivergingNorm(0)})


...

/home/xqiu/.local/lib/python3.6/site-packages/anndata/_core/anndata.py:1342: FutureWarning: In a future version of AnnData, access to `.X` by passing `layer='X'` will be removed. Instead pass `layer=None`.
  FutureWarning,


Saving figure to /home/xqiu/proj/Aristotle/data/u_penn_data/n_s_labeling_stream_velocity_filter_cells_dyn_savefig.svg...
Done


/home/xqiu/.local/lib/python3.6/site-packages/anndata/_core/anndata.py:1342: FutureWarning: In a future version of AnnData, access to `.X` by passing `layer='X'` will be removed. Instead pass `layer=None`.
  FutureWarning,


Saving figure to /home/xqiu/proj/Aristotle/data/u_penn_data/n_s_labeling_stream_velocity_legend_filter_cells_dyn_savefig.svg...
Done


<Figure size 432x288 with 0 Axes>

In [26]:
hao_gene_list = ["Egr1", "Fos", "Fosb", "Ier2", "Jun", "Nr4a3", "Homer1", "Fosl2", "Nefl", "Maff"]
dyn.pl.phase_portraits(adata, genes=hao_gene_list, color='time',  discrete_continous_div_color_key=[None, None, None], discrete_continous_div_color_key_cmap=['viridis', None, None], ncols=6, save_kwargs={'prefix': 'hao_gene_list_pp'}, save_show_or_return='save', pointsize=5)


/home/xqiu/.local/lib/python3.6/site-packages/dynamo-0.0+9871d78-py3.6.egg/dynamo/plot/dynamics.py:174: MatplotlibDeprecationWarning: 
The DivergingNorm class was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use TwoSlopeNorm instead.
  div_scatter_kwargs.update({"norm": DivergingNorm(0)})


Saving figure to /home/xqiu/proj/Aristotle/data/u_penn_data/hao_gene_list_pp_dyn_savefig.pdf...
Done


In [27]:
adata[:, hao_gene_list].var

,gene_short_name,pass_basic_filter,use_for_dynamo,alpha,beta,gamma,half_life,alpha_b,alpha_r2,gamma_b,...,uu0,ul0,su0,sl0,U0,S0,total0,beta_k,gamma_k,use_for_velocity
Egr1,Egr1,True,True,3.754236,None,0.564103,1.228760,None,None,0,...,None,None,None,None,None,None,None,None,0.676387,True
Fos,Fos,True,True,3.945576,None,0.604691,1.146283,None,None,0,...,None,None,None,None,None,None,None,None,0.701618,True
Fosb,Fosb,True,True,1.503470,None,0.375635,1.845267,None,None,0,...,None,None,None,None,None,None,None,None,0.528233,True
Ier2,Ier2,True,True,1.004515,None,0.451171,1.536328,None,None,0,...,None,None,None,None,None,None,None,None,0.594382,True
Jun,Jun,True,True,0.661064,None,0.608166,1.139734,None,None,0,...,None,None,None,None,None,None,None,None,0.703685,True
Nr4a3,Nr4a3,True,True,1.968013,None,0.540665,1.282027,None,None,0,...,None,None,None,None,None,None,None,None,0.660856,True
Homer1,Homer1,True,True,0.672634,None,0.488364,1.419324,None,None,0,...,None,None,None,None,None,None,None,None,0.623459,True
Fosl2,Fosl2,True,True,0.502122,None,0.527234,1.314687,None,None,0,...,None,None,None,None,None,None,None,None,0.651622,True
Nefl,Nefl,True,True,0.159578,None,0.135433,5.118020,None,None,0,...,None,None,None,None,None,None,None,None,0.237281,True
Maff,Maff,True,True,0.246222,None,1.767510,0.392160,None,None,0,...,None,None,None,None,None,None,None,None,0.970842,False


In [28]:
adata[:, hao_gene_list].var.loc[:, 'gamma_r2']

Egr1      0.825451
Fos       0.914950
Fosb      0.893733
Ier2      0.766159
Jun       0.694072
Nr4a3     0.760848
Homer1    0.837653
Fosl2     0.601055
Nefl      0.797660
Maff     -1.366342
Name: gamma_r2, dtype: float64

In [29]:
adata

AnnData object with n_obs × n_vars = 3066 × 97 
    obs: 'nGene', 'nUMI', 'time', 'cluster', 'label_time', 'use_for_dynamo', 'Size_Factor', 'initial_cell_size', 'total_Size_Factor', 'initial_total_cell_size', 'new_Size_Factor', 'initial_new_cell_size'
    var: 'gene_short_name', 'pass_basic_filter', 'use_for_dynamo', 'alpha', 'beta', 'gamma', 'half_life', 'alpha_b', 'alpha_r2', 'gamma_b', 'gamma_r2', 'gamma_logLL', 'delta_b', 'delta_r2', 'uu0', 'ul0', 'su0', 'sl0', 'U0', 'S0', 'total0', 'beta_k', 'gamma_k', 'use_for_velocity'
    uns: 'pp_log', 'PCs', 'explained_variance_ratio_', 'pca_fit', 'feature_selection', 'moments_con', 'dynamics', 'neighbors', 'umap_fit', 'velocity_pca_fit', 'velocity_PCs', 'kmc', 'transition_matrix', 'grid_velocity_umap', 'transition_matrix_rnd', 'grid_velocity_umap_rnd'
    obsm: 'X_pca', 'X_umap', 'X_velocity_pca', 'velocity_pca_raw', 'velocity_umap', 'X_umap_rnd', 'velocity_umap_rnd'
    varm: 'alpha'
    layers: 'new', 'total', 'X_total', 'X_new', 'M_t', 'M

In [30]:
adata.obs['umap_0'] = adata.obsm['X_umap'][:, 0]
adata.obs['umap_1'] = adata.obsm['X_umap'][:, 1]


In [31]:
adata.obs.to_csv('adata_NTR_obs_2k_genes.csv')

In [32]:
adata

AnnData object with n_obs × n_vars = 3066 × 97 
    obs: 'nGene', 'nUMI', 'time', 'cluster', 'label_time', 'use_for_dynamo', 'Size_Factor', 'initial_cell_size', 'total_Size_Factor', 'initial_total_cell_size', 'new_Size_Factor', 'initial_new_cell_size', 'umap_0', 'umap_1'
    var: 'gene_short_name', 'pass_basic_filter', 'use_for_dynamo', 'alpha', 'beta', 'gamma', 'half_life', 'alpha_b', 'alpha_r2', 'gamma_b', 'gamma_r2', 'gamma_logLL', 'delta_b', 'delta_r2', 'uu0', 'ul0', 'su0', 'sl0', 'U0', 'S0', 'total0', 'beta_k', 'gamma_k', 'use_for_velocity'
    uns: 'pp_log', 'PCs', 'explained_variance_ratio_', 'pca_fit', 'feature_selection', 'moments_con', 'dynamics', 'neighbors', 'umap_fit', 'velocity_pca_fit', 'velocity_PCs', 'kmc', 'transition_matrix', 'grid_velocity_umap', 'transition_matrix_rnd', 'grid_velocity_umap_rnd'
    obsm: 'X_pca', 'X_umap', 'X_velocity_pca', 'velocity_pca_raw', 'velocity_umap', 'X_umap_rnd', 'velocity_umap_rnd'
    varm: 'alpha'
    layers: 'new', 'total', 'X_total'

In [33]:
!pwd

/home/xqiu/proj/Aristotle/data/u_penn_data


In [ ]:
adata = neuron_splicing_4_11.copy()

n_top_genes=1001 # this is larger than 97 neuronal activity genes, all genes will thus be selected for pca 
time = ['0', '15', '30', '60', '120'] # , '60', '120'
cell_type=['Ex']

peng_gene_list = pd.read_csv('/home/xqiu/proj/Aristotle/data/u_penn_data/scNT_revision/0408_grp_info.txt', sep='\t')

model = 'stochastic'
tkey = 'time'
if cell_type is not None: adata = adata[adata.obs.cluster.isin(cell_type), :]

adata.obs['label_time'] = 2 # this is the labeling time 
tkey = 'label_time'
adata = adata[adata.obs.time.isin(time), :]

cell_filter = (adata.layers['total'] > 0).sum(1) > 2000

adata = adata[cell_filter.A1, :]

adata = adata[:, peng_gene_list.index]

adata

adata = dyn.pp.recipe_monocle(adata, n_top_genes=n_top_genes, fc_kwargs={"min_expr_genes_s": 5,
    "min_expr_genes_u": 2,}, fg_kwargs={"shared_count": 5},
#                       keep_filtered_cells=False,
#                       keep_filtered_genes=False,
                      # total_layers='spliced',
                      )

dyn.tl.dynamics(adata, model=model, tkey=tkey)

dyn.tl.reduceDimension(adata, reduction_method='umap')

dyn.tl.cell_velocities(adata, calc_rnd_vel=True)
adata.obs['time'] = adata.obs.time.astype('category')

dyn.pl.phase_portraits(adata, genes=peng_gene_list.index[0:25], color='time', discrete_continous_div_color_key_cmap=['viridis', None, None], save_show_or_return='save', pointsize=3, save_kwargs={'prefix': 'labeling_pp_0_25_filter_cells'})
dyn.pl.phase_portraits(adata, genes=peng_gene_list.index[25:50], color='time', discrete_continous_div_color_key_cmap=['viridis', None, None], save_show_or_return='save', pointsize=3, save_kwargs={'prefix': 'labeling_pp_25_50_filter_cells'})
dyn.pl.phase_portraits(adata, genes=peng_gene_list.index[50:75], color='time', discrete_continous_div_color_key_cmap=['viridis', None, None], save_show_or_return='save', pointsize=3, save_kwargs={'prefix': 'labeling_pp_50_75_filter_cells'})
dyn.pl.phase_portraits(adata, genes=peng_gene_list.index[75:], color='time', discrete_continous_div_color_key_cmap=['viridis', None, None], save_show_or_return='save', pointsize=3, save_kwargs={'prefix': 'labeling_pp_75_above_filter_cells'})

hao_gene_list = ["Egr1", "Fos", "Fosb", "Ier2", "Jun", "Nr4a3", "Homer1", "Fosl2", "Nefl", "Maff"]
dyn.pl.phase_portraits(adata, genes=hao_gene_list, color='time',  discrete_continous_div_color_key=[None, None, None], discrete_continous_div_color_key_cmap=['viridis', None, None], ncols=6, save_kwargs={'prefix': 'hao_gene_list_pp'}, save_show_or_return='save', pointsize=5)

dyn.pl.cell_wise_velocity(adata, color='time', color_key_cmap = 'viridis', basis='umap', ncols=5, quiver_size=4,
                          quiver_length=5, save_show_or_return='save', save_kwargs={'prefix': 'labeling_cell_wise_velocity_filter_cells'})  # ['GRIA3', 'LINC00982', 'AFF2']

dyn.pl.grid_velocity(adata, color='time', color_key_cmap = 'viridis', basis='umap', ncols=5, save_show_or_return='save', save_kwargs={'prefix': 'labeling_grid_velocity_filter_cells'})  # ['GRIA3', 'LINC00982', 'AFF2']
dyn.pl.streamline_plot(adata, color='time', color_key_cmap = 'viridis', basis='umap_rnd', ncols=5, show_legend='on data', save_show_or_return='save', save_kwargs={'prefix': 'labeling_streamline_rnd_filter_cells', 'ext': 'svg'})  # ['GRIA3', 'LINC00982', 'AFF2']
dyn.pl.streamline_plot(adata, color='time', color_key_cmap = 'viridis', basis='umap_rnd', ncols=5, save_show_or_return='save', save_kwargs={'prefix': 'labeling_streamline_rnd_legend_filter_cells', 'ext': 'svg'})  # ['GRIA3', 'LINC00982', 'AFF2']
